<a href="https://colab.research.google.com/github/BhoomiMahna/outfit-vibe-rater/blob/main/complexity_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving outfits_final.zip to outfits_final.zip


In [ ]:
import os
import zipfile

zip_path='outfits_final.zip'
extract_path='outfits_final'

if not os.path.exists(extract_path):
  with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
    print(f"Files extracted to '{extract_path}' folder.")
else:
    print("Files already extracted.")

Files extracted to 'outfits_final' folder.


In [ ]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import models,layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

data_dir ='outfits_final/outfits/complexity'
img_size=(260,260)
batch_size=32
model_save_path ="complexity_final.h5"


train_ds=keras.utils.image_dataset_from_directory(
    data_dir,
    label_mode='categorical',
    validation_split=0.2,
    subset='training',
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)
val_ds=tf.keras.utils.image_dataset_from_directory(
    data_dir,
    label_mode='categorical',
    validation_split=0.2,
    subset='validation',
    seed=42,
    image_size=img_size,
    batch_size=batch_size

)

class_names = train_ds.class_names
num_classes=len(class_names)
print(f"found {num_classes} classes: {class_names}")

Found 830 files belonging to 2 classes.
Using 664 files for training.
Found 830 files belonging to 2 classes.
Using 166 files for validation.
found 2 classes: ['multiple layer', 'single layer']


In [ ]:
data_augmentation=models.Sequential([
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
], name="data_augmentation" )

AUTOTUNE=tf.data.AUTOTUNE
train_ds=train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds=val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:

base_model = tf.keras.applications.EfficientNetV2B2(
    input_shape=img_size + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

inputs = layers.Input(shape=img_size + (3,))
x = data_augmentation(inputs)
x = tf.keras.applications.efficientnet_v2.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = models.Model(inputs, outputs)
print("Model built with EfficientNetV2B2 base.")



initial_epochs = 10
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n Starting Initial Training (Training the head)")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=initial_epochs
)



base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


checkpoint_cb = ModelCheckpoint(
    model_save_path,
    save_best_only=True,
    monitor="val_accuracy",
    mode="max"
)
early_stopping_cb = EarlyStopping(
    patience=3,
    monitor="val_loss",
    restore_best_weights=True
)
print("Model re-compiled for fine-tuning.")



print("\n Starting Fine-Tuning")
fine_tune_epochs = 15
total_epochs = initial_epochs + fine_tune_epochs

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    callbacks=[checkpoint_cb, early_stopping_cb]
)



print("\nEvaluating Final Model ")

best_model = tf.keras.models.load_model(model_save_path)
final_loss, final_accuracy = best_model.evaluate(val_ds)
print(f"\nFinal Validation Accuracy for Pattern Density Model: {final_accuracy:.4f}")

35839040/35839040 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Model built with EfficientNetV2B2 base.

 Starting Initial Training (Training the head)
Epoch 1/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 30s 499ms/step - accuracy: 0.5402 - loss: 0.7695 - val_accuracy: 0.8072 - val_loss: 0.5178
Epoch 2/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 132ms/step - accuracy: 0.7076 - loss: 0.5642 - val_accuracy: 0.8614 - val_loss: 0.4163
Epoch 3/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 137ms/step - accuracy: 0.7356 - loss: 0.5224 - val_accuracy: 0.8675 - val_loss: 0.3750
Epoch 4/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 131ms/step - accuracy: 0.7407 - loss: 0.5133 - val_accuracy: 0.8675 - val_loss: 0.3449
Epoch 5/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 137ms/step - accuracy: 0.8005 - loss: 0.4452 - val_accuracy: 0.8795 - val_loss: 0.3240
Epoch 6/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 138ms/step - accuracy: 0.7690 - loss: 0.4544 - val_accuracy: 0.8735 - val_loss: 0.3197
Epoch 7/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 134ms/step - accuracy: 0.7867 - loss: 0.4590 - val_

21/21 ━━━━━━━━━━━━━━━━━━━━ 29s 428ms/step - accuracy: 0.7744 - loss: 0.4862 - val_accuracy: 0.9157 - val_loss: 0.3059
Epoch 11/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 166ms/step - accuracy: 0.8168 - loss: 0.4506 - val_accuracy: 0.9096 - val_loss: 0.3129
Epoch 12/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 174ms/step - accuracy: 0.8338 - loss: 0.4226 - val_accuracy: 0.9096 - val_loss: 0.3083
Epoch 13/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 174ms/step - accuracy: 0.8037 - loss: 0.4451 - val_accuracy: 0.9157 - val_loss: 0.3018
Epoch 14/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 175ms/step - accuracy: 0.7899 - loss: 0.4511 - val_accuracy: 0.9157 - val_loss: 0.2964
Epoch 15/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 184ms/step - accuracy: 0.8136 - loss: 0.4266 - val_accuracy: 0.9157 - val_loss: 0.2919
Epoch 16/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.8329 - loss: 0.4307

21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 204ms/step - accuracy: 0.8331 - loss: 0.4303 - val_accuracy: 0.9217 - val_loss: 0.2869
Epoch 17/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 176ms/step - accuracy: 0.8370 - loss: 0.4139 - val_accuracy: 0.9217 - val_loss: 0.2788
Epoch 18/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 178ms/step - accuracy: 0.8401 - loss: 0.4021 - val_accuracy: 0.9217 - val_loss: 0.2756
Epoch 19/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 176ms/step - accuracy: 0.8170 - loss: 0.4149 - val_accuracy: 0.9217 - val_loss: 0.2720
Epoch 20/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 177ms/step - accuracy: 0.8352 - loss: 0.4092 - val_accuracy: 0.9217 - val_loss: 0.2693
Epoch 21/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 180ms/step - accuracy: 0.8775 - loss: 0.3686 - val_accuracy: 0.9157 - val_loss: 0.2640
Epoch 22/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 181ms/step - accuracy: 0.8454 - loss: 0.3925 - val_accuracy: 0.9217 - val_loss: 0.2605
Epoch 23/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 177ms/step - accuracy: 0.8327 - loss: 0.3815 - val_accuracy: 0.9217

6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 93ms/step - accuracy: 0.9282 - loss: 0.2749 

Final Validation Accuracy for Pattern Density Model: 0.9217
